In [ ]:
#@title 🔧 OpenRouter API Key Management Tool { display-mode: "form" }
#@markdown ### Welcome! This tool helps you manage your OpenRouter API keys in bulk.
#@markdown
#@markdown **📋 What you need:**
#@markdown - A provisioning API key from [OpenRouter](https://openrouter.ai/settings/provisioning-keys) (store it in Colab Secrets for security)
#@markdown
#@markdown **🚀 How to use:**
#@markdown 1. Run this cell
#@markdown 2. Click "Load Keys" to fetch all your API keys
#@markdown 3. Review the keys in the interactive table
#@markdown 4. Check boxes next to keys you want to delete
#@markdown 5. Click "Delete Checked Keys" to remove selected keys
#@markdown
#@markdown **🔐 Security Tip:** Use Colab Secrets for your provisioning key (`OPENROUTER_PROVISIONING_KEY`).
#@markdown
#@markdown **⚠️ Warning:** Deleted keys cannot be recovered. Use with caution!
#@markdown
#@markdown ---

# Install and import required libraries
import sys

# Install dependencies
try:
    import pandas as pd
except ImportError:
    !pip install -q pandas
    import pandas as pd

try:
    import ipywidgets as widgets
except ImportError:
    !pip install -q ipywidgets
    import ipywidgets as widgets

# Standard library imports
import os
import time
import requests
from datetime import datetime
from IPython.display import display, HTML, clear_output

# Check if running in Google Colab
try:
    from google.colab import userdata, output
    IN_COLAB = True
    # Enable widget manager for Colab
    output.enable_custom_widget_manager()
except ImportError:
    IN_COLAB = False
    userdata = None

print("✅ All libraries loaded successfully\n")

# ============================================================================
# CORE FUNCTIONS
# ============================================================================

def list_all_keys(provision_key):
    """Fetch all API keys from OpenRouter using pagination."""
    BASE_URL = "https://openrouter.ai/api/v1/keys"
    all_keys = []
    offset = 0
    limit = 100
    
    try:
        while True:
            response = requests.get(
                url=BASE_URL,
                headers={
                    "Authorization": f"Bearer {provision_key}",
                    "Content-Type": "application/json"
                },
                params={"offset": offset, "limit": limit},
                timeout=30
            )
            
            if response.status_code == 200:
                data = response.json()
                keys = data.get('data', [])
                if not keys:
                    break
                all_keys.extend(keys)
                if len(keys) < limit:
                    break
                offset += limit
            else:
                return None, f"HTTP {response.status_code}: {response.text}"
        
        return all_keys, None
    except Exception as e:
        return None, str(e)


def delete_key(provision_key, key_hash):
    """Delete a single API key by its hash."""
    BASE_URL = f"https://openrouter.ai/api/v1/keys/{key_hash}"
    try:
        response = requests.delete(
            url=BASE_URL,
            headers={
                "Authorization": f"Bearer {provision_key}",
                "Content-Type": "application/json"
            },
            timeout=30
        )
        
        if response.status_code in [200, 204]:
            return True, None
        else:
            return False, f"HTTP {response.status_code}: {response.text}"
    except Exception as e:
        return False, str(e)


def format_datetime(dt_string):
    """Format ISO datetime string to readable format."""
    try:
        dt = datetime.fromisoformat(dt_string.replace('Z', '+00:00'))
        return dt.strftime('%Y-%m-%d %H:%M UTC')
    except:
        return dt_string


def format_usage(usage):
    """Format usage amount in dollars."""
    try:
        return f"${float(usage):.2f}"
    except:
        return str(usage)

# ============================================================================
# GUI CREATION
# ============================================================================

provisioning_key_from_secrets = None
if IN_COLAB and userdata is not None:
    try:
        provisioning_key_from_secrets = userdata.get('OPENROUTER_PROVISIONING_KEY')
    except Exception:
        provisioning_key_from_secrets = None

if provisioning_key_from_secrets:
    provision_key_widget = widgets.Text(
        value='',
        placeholder='✅ PROVISIONING KEY LOADED - GOOD TO GO',
        description='Provision Key:',
        disabled=True,
        style={'description_width': '120px'},
        layout=widgets.Layout(width='600px')
    )
else:
    provision_key_widget = widgets.Password(
        value='',
        placeholder='⚠️ ENTER YOUR OWN PROVISIONING KEY',
        description='Provision Key:',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='600px')
    )

load_button = widgets.Button(
    description='🔄 Load Keys',
    button_style='info',
    layout=widgets.Layout(width='200px', height='42px'),
    style={'font_weight': 'bold'}
)

delete_button = widgets.Button(
    description='🗑️ Delete Checked Keys',
    button_style='danger',
    layout=widgets.Layout(width='200px', height='42px'),
    style={'font_weight': 'bold'},
    disabled=True
)

select_all_button = widgets.Button(
    description='☑️ Select All',
    button_style='',
    layout=widgets.Layout(width='120px'),
    disabled=True
)

deselect_all_button = widgets.Button(
    description='⬜ Deselect All',
    button_style='',
    layout=widgets.Layout(width='120px'),
    disabled=True
)

output_area = widgets.Output()
table_container = widgets.VBox()

# Global state
keys_data = []
checkboxes = []

# ============================================================================
# EVENT HANDLERS
# ============================================================================

def on_load_click(button):
    global keys_data, checkboxes
    
    with output_area:
        clear_output()
        provision_key = provisioning_key_from_secrets or provision_key_widget.value
        
        if not provision_key or provision_key.strip() == '':
            print("❌ ERROR: Provisioning key is required.")
            return
        
        print("⏳ Loading API keys...\n")
        keys, error = list_all_keys(provision_key)
        
        if error:
            print(f"❌ ERROR: {error}")
            return
        
        if not keys:
            print("ℹ️ No API keys found.")
            keys_data = []
            checkboxes = []
            table_container.children = []
            delete_button.disabled = True
            select_all_button.disabled = True
            deselect_all_button.disabled = True
            return
        
        keys_data = keys
        print(f"✅ Loaded {len(keys_data)} API keys\n")
        
        # Create interactive table
        checkboxes = []
        table_rows = []
        
        # Header row
        header = widgets.HBox([
            widgets.Label(value='Delete', layout=widgets.Layout(width='60px')),
            widgets.Label(value='Name', layout=widgets.Layout(width='280px')),
            widgets.Label(value='Created', layout=widgets.Layout(width='180px')),
            widgets.Label(value='Usage', layout=widgets.Layout(width='100px')),
            widgets.Label(value='Limit', layout=widgets.Layout(width='100px')),
            widgets.Label(value='Status', layout=widgets.Layout(width='100px')),
        ], layout=widgets.Layout(border='2px solid #333', padding='8px', margin='0 0 5px 0'))
        table_rows.append(header)
        
        # Data rows
        for i, key in enumerate(keys_data):
            checkbox = widgets.Checkbox(
                value=False,
                indent=False,
                layout=widgets.Layout(width='60px', margin='0')
            )
            checkboxes.append(checkbox)
            
            name = key.get('name', 'N/A')
            created = format_datetime(key.get('created_at', 'N/A'))
            usage = format_usage(key.get('usage', 0))
            limit = f"${key.get('limit', 0):.2f}" if key.get('limit') else 'N/A'
            disabled = key.get('disabled', False)
            status = '🔴 Disabled' if disabled else '🟢 Active'
            
            # Truncate long names
            display_name = name[:35] + '...' if len(name) > 35 else name
            
            row = widgets.HBox([
                checkbox,
                widgets.Label(value=display_name, layout=widgets.Layout(width='280px')),
                widgets.Label(value=created, layout=widgets.Layout(width='180px')),
                widgets.Label(value=usage, layout=widgets.Layout(width='100px')),
                widgets.Label(value=limit, layout=widgets.Layout(width='100px')),
                widgets.Label(value=status, layout=widgets.Layout(width='100px')),
            ], layout=widgets.Layout(border='1px solid #ddd', padding='4px', margin='2px 0'))
            table_rows.append(row)
        
        # Update table container
        table_container.children = table_rows
        
        # Enable buttons
        delete_button.disabled = False
        select_all_button.disabled = False
        deselect_all_button.disabled = False
        
        # Show summary
        total_usage = sum(key.get('usage', 0) for key in keys_data)
        active_count = sum(1 for key in keys_data if not key.get('disabled', False))
        disabled_count = len(keys_data) - active_count
        
        print("📊 SUMMARY")
        print("=" * 40)
        print(f"Total keys: {len(keys_data)}")
        print(f"Active: {active_count} | Disabled: {disabled_count}")
        print(f"Total usage: ${total_usage:.2f}")
        print("=" * 40)
        print("\n💡 TIP: Check the boxes in the table above to select keys for deletion.")


def on_delete_click(button):
    global keys_data, checkboxes
    
    with output_area:
        clear_output()
        provision_key = provisioning_key_from_secrets or provision_key_widget.value
        
        if not provision_key or provision_key.strip() == '':
            print("❌ ERROR: Provisioning key is required.")
            return
        
        # Get selected keys
        selected_indices = [i for i, cb in enumerate(checkboxes) if cb.value]
        
        if not selected_indices:
            print("ℹ️ No keys selected for deletion.")
            return
        
        selected_keys = [keys_data[i] for i in selected_indices]
        
        print(f"⚠️ WARNING: About to delete {len(selected_keys)} key(s)\n")
        print("Keys to be deleted:")
        for key in selected_keys:
            print(f"  • {key.get('name', 'N/A')}")
        print("\n⏳ Deleting keys...\n")
        
        successful = 0
        failed = 0
        errors = []
        
        for key in selected_keys:
            key_hash = key.get('hash')
            key_name = key.get('name', 'N/A')
            
            if not key_hash:
                failed += 1
                errors.append({'name': key_name, 'error': 'Missing key hash'})
                continue
            
            success, error = delete_key(provision_key, key_hash)
            
            if success:
                successful += 1
                print(f"✅ Deleted: {key_name}")
            else:
                failed += 1
                errors.append({'name': key_name, 'error': error})
                print(f"❌ Failed: {key_name} - {error}")
            
            time.sleep(0.1)  # Rate limiting
        
        print("\n" + "=" * 60)
        print("📊 DELETION RESULTS")
        print("=" * 60)
        print(f"✅ Successful: {successful} / {len(selected_keys)}")
        print(f"❌ Failed: {failed} / {len(selected_keys)}")
        print("=" * 60)
        
        if errors:
            print("\n⚠️ ERRORS:")
            for err in errors:
                print(f"  • {err['name']}: {err['error']}")
        
        if successful > 0:
            print("\n💡 TIP: Click 'Load Keys' to refresh the table.")


def on_select_all_click(button):
    for cb in checkboxes:
        cb.value = True


def on_deselect_all_click(button):
    for cb in checkboxes:
        cb.value = False

# Attach handlers
load_button.on_click(on_load_click)
delete_button.on_click(on_delete_click)
select_all_button.on_click(on_select_all_click)
deselect_all_button.on_click(on_deselect_all_click)

# ============================================================================
# DISPLAY GUI
# ============================================================================
print("🎨 OpenRouter API Key Management Interface\n")

key_box = widgets.VBox([
    widgets.HTML('<h3>🔑 Step 1: Provisioning Key</h3>'),
    provision_key_widget
])

load_box = widgets.VBox([
    widgets.HTML('<h3>📋 Step 2: Load Keys</h3>'),
    load_button
])

table_box = widgets.VBox([
    widgets.HTML('<h3>🗂️ Step 3: Review & Select Keys</h3>'),
    widgets.HBox([select_all_button, deselect_all_button]),
    widgets.HTML('<div style="height:10px"></div>'),
    table_container
])

delete_box = widgets.VBox([
    widgets.HTML('<h3>🗑️ Step 4: Delete Keys</h3>'),
    widgets.HTML('<p style="color: #d9534f; font-size: 0.9em;">⚠️ Warning: Deletion is permanent and cannot be undone!</p>'),
    delete_button
])

ui = widgets.VBox([
    key_box,
    load_box,
    table_box,
    delete_box,
    widgets.HTML('<hr>'),
    output_area
])

display(ui)